# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [21]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [22]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


bio_template = """你是一位很棒的生物家。你擅长回答生物问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


cs_template = """你是一位很棒的计算机领域人员。你擅长回答计算机领域相关问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


cn_template = """你是一位很棒的语文文学家家。你擅长回答语文问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

In [23]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": bio_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": cs_template,
    },
    {
        "name": "汉语言文学老师",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": cn_template,
    },
]

In [24]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [25]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [26]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [27]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [28]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [29]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学老师: 适用于回答汉语言文学问题']


In [30]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学老师: 适用于回答汉语言文学问题


In [31]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [32]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学老师: 适用于回答汉语言文学问题

<< INPUT >>
{input}


In [33]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [34]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指处于热平衡状态的物体所发出的电磁辐射。根据普朗克定律，这种辐射的强度与物体的温度有关，而与物体的化学组成、形状等无关。黑体辐射的频谱与温度有关，当温度升高时，辐射的频谱会向高频方向移动，即辐射的波长会变短。黑体辐射是理解热力学和量子力学的重要概念，也是研究太阳、行星等天体的辐射特性的基础。'}


In [35]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '40'}
> Finished chain.
{'input': '40', 'text': 'x50\n\n首先，我们可以将40和50分别分解为2个乘数，即40=8x5，50=10x5。\n然后，我们可以使用乘法的结合律，将问题重写为：\n(8x5)x(10x5)。\n\n接下来，我们可以使用乘法的交换律，将问题重写为：\n(8x10)x(5x5)。\n\n最后，我们可以使用乘法的结合律，将问题重写为：\n(8x10)x(25)。\n\n因此，答案为2000。你真的是一位很棒的数学家，能够将问题分解并使用数学规律解决它。继续努力，你一定会有更多的数学成就！'}


In [36]:
router_chain.verbose = True

In [37]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是一种非常密集的天体，它的质量非常大而体积非常小，以至于它的引力非常强大，甚至连光都无法逃脱。由于它的引力非常强大，所以它吸引周围的物质，形成一个巨大的漩涡。黑洞的存在是由爱因斯坦的广义相对论预言的，它是宇宙中最神秘的天体之一，也是天文学研究中的一个重要课题。'}


In [38]:
print(chain.invoke("你说一下有丝分裂的过程？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '你说一下有丝分裂的过程？'}
> Finished chain.
{'input': '你说一下有丝分裂的过程？', 'text': '\n有丝分裂是一种细胞分裂方式，可以将一个细胞分裂成两个完全相同的细胞。它分为四个连续的阶段：前期、中期、后期和末期。\n\n在前期，细胞的染色质会变得更加紧密，形成染色体。每个染色体包含两条相同的染色单体，称为姊妹染色体。此时，细胞的中心体会开始分裂，形成两个紧密排列的中心粒子。\n\n接下来是中期，此时细胞的核壁开始消失，染色体向细胞的中心移动。同时，纺锤体（一种细胞器）会在细胞的两端形成，它们的纤维会与染色体连接。\n\n在后期，纺锤体的纤'}


In [42]:
print(chain.invoke("汉语言文学是什么"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学老师: {'input': '汉语言文学是什么'}
> Finished chain.
{'input': '汉语言文学是什么', 'text': '？\n\n汉语言文学是指使用汉语作为表达工具，通过文字、诗歌、散文、小说等文学形式，反映中国人民的思想、生活和情感的艺术。它包括古代和现代的文学作品，是中国传统文化的重要组成部分。汉语言文学也涉及到语言的使用和表现技巧，包括词汇、语法、修辞等方面。总的来说，汉语言文学是中国人民用汉语来表达自己的文学艺术。'}


In [39]:
print(chain.invoke("C#可以用来干什么"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': 'C#可以用来干什么'}
> Finished chain.
{'input': 'C#可以用来干什么', 'text': '？\n\nC#是一种通用的面向对象编程语言，它可以用来开发各种类型的软件应用程序。它具有强大的类型安全性、可扩展性和可移植性，使它成为开发桌面应用程序、Web应用程序、移动应用程序和游戏等各种类型应用程序的理想选择。C#也被广泛应用于大型企业级应用程序，以及云计算和人工智能等最新技术领域。总的来说，C#可以用来实现各种复杂的计算机程序和系统，满足不同行业和领域的需求。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains